In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('final_rpl.csv', sep=';', index_col = 0)
df.head()

,Команда_1,Голы_1,Команда_2,Голы_2,xG_1,xG_2,Удары_1,Удары_2,Удары в створ_1,Удары в створ_2,...,Время,Кэф_1,Кэф_х,Кэф_2,Градусы,Погода,Зрители,Дома_1,Дома_2,Таргет
0,Ростов,2,Оренбург,1,1.67,1.10,23.0,11.0,7.0,4.0,...,12:45:00,2.36,3.41,3.34,19,переменная облачность,14680.0,1,0,1
1,ПФК Сочи,0,Факел,0,1.21,0.51,22.0,5.0,3.0,2.0,...,10:30:00,2.15,3.18,4.25,15,небольшой дождь,5012.0,1,0,0
2,Рубин,1,Урал,1,0.85,1.30,5.0,12.0,3.0,4.0,...,08:15:00,2.59,3.01,3.37,6,переменная облачность,8178.0,1,0,0
3,Крылья Советов,0,Краснодар,0,1.11,0.90,14.0,12.0,6.0,3.0,...,06:00:00,3.51,3.42,2.28,6,переменная облачность,9689.0,1,0,0
4,Балтика,3,ЦСКА Москва,1,1.88,1.14,13.0,18.0,4.0,7.0,...,14:30:00,2.66,3.41,2.89,20,солнечно,14046.0,1,0,1


In [12]:
df.shape

(208, 54)

как видно у нас сейчас достаточно маленький датасет. Сейчас осталось сделать пару штрихов, как мы помним, нам осталось преобразовать только нашу Погоду и Команды. Для этого отлично подойдет one hot encoding. Но у нас так же есть подводный камень, дело в том, что наши даты и время все еще в непригодном для обучения состоянии. Давайте оставим от времени только часы, а колонку дата разобъем на три: год, меяц, день

In [5]:
df_1 = df.copy()
df_1 = pd.get_dummies(df_1, columns = ['Погода'])
df_1['Дата'] = pd.to_datetime(df_1['Дата'])
df_1['Год'] = df_1['Дата'].dt.year
df_1['Месяц'] = df_1['Дата'].dt.month
df_1['День'] = df_1['Дата'].dt.day
df_1 = df_1.drop('Дата', axis = 1)
df_1['Время'] = pd.to_timedelta(df_1['Время']).dt.components.hours

Ну все, теперь у нас есть красивый датасет, давайте обучим на нем несколько моделей и посмотрим на аккуратность каждой

In [6]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 208 entries, 0 to 207
Data columns (total 59 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Команда_1                     208 non-null    object 
 1   Голы_1                        208 non-null    int64  
 2   Команда_2                     208 non-null    object 
 3   Голы_2                        208 non-null    int64  
 4   xG_1                          208 non-null    float64
 5   xG_2                          208 non-null    float64
 6   Удары_1                       208 non-null    float64
 7   Удары_2                       208 non-null    float64
 8   Удары в створ_1               208 non-null    float64
 9   Удары в створ_2               208 non-null    float64
 10  Блок-но ударов_1              208 non-null    float64
 11  Блок-но ударов_2              208 non-null    float64
 12  Сейвы_1                       208 non-null    float64
 13  Сейвы_2   

Попробуем логистическую регрессию
Для этого закодируем последние признаки, разобъем на тестовую и обучающую выборки и немного все перемешаем.
Так же для улучшения сходимости и качества обучения данные лучше всего нормализовать, поэтому воспользуемся стандартным скалером.

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df_1_encoded = pd.get_dummies(df_1, columns=['Команда_1', 'Команда_2'])
X = df_1_encoded.drop('Таргет', axis = 1)
y = df_1_encoded['Таргет']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Обучим наше логистическую регрессию

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели: {accuracy:.2f}")

Теперь для сравнения обучим линейную регрессию с RMSE. Конечно, даже не обучая можно сказать, что это достаточно плохой выбор, потому что она предназначена для предсказания неприрывных значений, плюс признаков для нее многовато. Давайте в этом убедимся.

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

linear_model = LinearRegression()
linear_model.fit(X_train_scaled, y_train)
y_pred = linear_model.predict(X_test_scaled)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE модели: {rmse:.2f}")

RMSE модели: 0.99


In [10]:
y_pred_rounded = np.round(y_pred).astype(int)
accuracy = np.mean(y_pred_rounded == y_test)
print(f"Приближенная точность модели: {accuracy:.2f}")


Приближенная точность модели: 0.36


Теперь так же для сравнения посмотрим на результаты модели случайного леса. Возьмем, например, 100 обучающих деревьев и посмотрим на аккуратность. Перемешивать не будем во второй раз, мы уже сделали это раньше, когда подготавливали данные.

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model = RandomForestClassifier(n_estimators=100)  # n_estimators - количество деревьев
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(f'Точность модели: {accuracy:.2f}')


Точность модели: 0.71


Как видно из наших примеров, лучше всего с задачей справились наши деревья, логистическая регрессия показала достойный результат, а вот линейная регрессия совсем плоха.
Но и это не конечный результат, можно попробовать подобрать другие парметры для обучения, увеличить количество данных, обучить не на всех параметрах, по другому их располагать, протестировать другие модели и тд.